In [1]:
# Perform the initialization and imports
import sys
import pickle
import re
import os
import csv
import argparse
import math
import pprint

from string import ascii_lowercase
from collections import Counter, defaultdict

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from Bio import SeqIO, AlignIO
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC
from Bio.Seq import Seq
from Bio.Emboss.Applications import NeedleallCommandline

# Demand Python 3.
if sys.version_info[0] < 3:
    print("Python 3 is required, but you are using Python %i.%i.%i") % (
        sys.version_info[0], sys.version_info[1], sys.version_info[2])
    sys.exit(1)
    
# Retrieve the specific functions from ind and proteins.py
indels_path="/home/maya/InDelScanner"  # /PATH/TO/InDelScanner
if indels_path not in sys.path:
    sys.path.append(indels_path)
#from indels.ind import trim_read, findEnds, endMatch, findGap, gapAlign

from ipynb.fs.defs.Library_diversity import convert_variant_to_dict

os.chdir("/mnt/c/Users/Maya/Dropbox/mek_results")

with open('Remkes_protein.p', 'rb') as f:
    all_ref = pickle.load(f)
with open('Remkes_protein_low.p', 'rb') as f:
    low = pickle.load(f)

all_ref['mek']['low-v2'] = low['mek']['low-v2']

mek = {}
for fraction in ['high', 'med']:
    mek[fraction] = Counter(all_ref['mek'][fraction])
mek['low-t'] = Counter(all_ref['mek']['low']) + Counter(all_ref['mek']['low-v2'])

In [2]:
# Set general restrictions stemming from SpliMLib library design
aa_2 = ['A', 'Δ']
aa_12 = ['A','G','P','Y','D','K','M','V','I','L','F','W']
aa_13 = aa_12 +  ['Δ']
pos_aa = {'6': aa_12, '9': aa_12, '11': aa_12, '13': aa_12, '7a': aa_13, '8a': aa_2}

In [3]:
df_all = pd.DataFrame.from_dict(mek).fillna(0).sort_values(by=['high', 'med', 'low-t'], ascending=False).astype(int)
df_50p = df_all.loc[(df_all['high'] >= 50) & ((df_all['high']+df_all['med']) > 2*df_all['low-t'])]
df_20to50 = df_all.loc[(df_all['high'].isin(range(10,50))) & 
                       (df_all['high'] > df_all['med']) & 
                       ((df_all['high']+df_all['med']) > 5*df_all['low-t']) ]

df_pos = df_50p.append(df_20to50)
pos = df_pos.to_dict()

## Clustering: k-medoids and hiearchical

In order to perform hiearchical clustering, I need a dataframe where each column represents the AA present at one randomised position. Then, each column is a categorical variable and the columns can be compared with Gower dissimilarity, followed by K-medioid clustering or hiearchical clustering.

https://healthcare.ai/clustering-non-continuous-variables/

https://www.thinkdatascience.com/post/2019-12-16-introducing-python-package-gower/

https://www.researchgate.net/publication/272351873_NumPy_SciPy_Recipes_for_Data_Science_k-Medoids_Clustering

In [4]:
import gower
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram

In [5]:
active_ls = df_pos.index.tolist()
active = {short : convert_variant_to_dict(short) for short in active_ls}

In [6]:
valid_pos = ['6', '7a', '8a', '9', '11', '13']

data = {}
for short, m_to_pos in active.items():
    if len(m_to_pos) != len(valid_pos):
        continue
    else:
        data[short] = [m_to_pos[i] for i in valid_pos]

factors = pd.DataFrame.from_dict(data, orient='index', columns=valid_pos).reset_index(drop=False)

In [7]:
factors.head()

,index,6,7a,8a,9,11,13
0,6L/7aI/8aA/9L/11F/13M,L,I,A,L,F,M
1,6F/7aP/9W/11L/13M,F,P,Δ,W,L,M
2,6L/7aF/9L/11I/13I,L,F,Δ,L,I,I
3,6A/7aI/8aA/9L/11L/13I,A,I,A,L,L,I
4,6W/7aI/9F/11L/13V,W,I,Δ,F,L,V


In [8]:
factors.iloc[:, 1:].head()

,6,7a,8a,9,11,13
0,L,I,A,L,F,M
1,F,P,Δ,W,L,M
2,L,F,Δ,L,I,I
3,A,I,A,L,L,I
4,W,I,Δ,F,L,V


## The Gower Dissimilarity

The Gower dissimilarity measure falls between 0 and 1, where 0 indicates that two elements are identical, and 1 that they are different for that factor. The scores for multiple factors are calculated as an average of all factors, with correction factors for missing values. In the case of this dataset, where there are no missing values and all data is categorical (where the Gower scoring essentially works out to Hamett distance), the scoring is a s follows:

comparing two variants $i$ and $j$, with property columns $c$ (c ranges from 1 to 6, for six randomised positions), without missing values:

$d_{(i,j)} = \frac{\sum_{c=1}^{6} \delta_{i,j}}{6}$

where $\delta_{i,j} = 0$ if two variants have the same amino acid at that position, and 1 otherwise. Thus, this distance is equivalent to scaled Hamett distance, so that it ranges between 0 and 1. Finally, the matrix is symmetric across the diagonal.

In [9]:
# gower_matrix = gower.gower_matrix(factors.iloc[:, 1:])

In [10]:
# np.savez_compressed('Full_gower_matrix.npz', gw=gower_matrix)

In [11]:
gower_matrix = np.load('Full_gower_matrix.npz')['gw']

In [12]:
gower_matrix.shape

(32375, 32375)

In [13]:
gower_df = pd.DataFrame(data=gower_matrix, index=factors.index, columns=list(factors.index))

This pre-computed matrix can then be used to update medoids and re-calculate distances; the calculation of distances within a cluster is reduced to looking up the distances in the master matrix.

Hiearchical clustering turns out to be computationally infeasible, for now. K-medoids solves the computational issues by using a pre-computed matrix of distances between variants.

In [14]:
def kMedoids(D, k, tmax=100):
    # determine dimensions of distance matrix D
    m, n = D.shape
    
    # randomly initialize an array of k medoid indices
    M = np.sort(np.random.choice(n, k))
    
    # create a copy of the array of medoid indices
    Mnew = np.copy(M)
    
    # initialize a dictionary to represent clusters
    C = {}
    
    for t in range(tmax): # t iterates over the convergence cycles
        # determine clusters, i.e. arrays of data indices
        J = np.argmin(D[:,M], axis=1)
        for kappa in range(k):
            C[kappa] = np.where(J==kappa)[0]
        
        # update cluster medoids
        for kappa in range(k):
            J = np.mean(D[np.ix_(C[kappa],C[kappa])],axis=1)
            j = np.argmin(J)
            Mnew[kappa] = C[kappa][j]
            np.sort(Mnew)
                
        # check for convergence
        if np.array_equal(M, Mnew):
            print(k, t)
            break
                
        M = np.copy(Mnew)
    
    else:
        # final update of cluster memberships
        print(k, t)
        J = np.argmin(D[:,M], axis=1)
        for kappa in range(k):
            C[kappa] = np.where(J==kappa)[0]
                
    # return results
    return M, C

The output of this algorithm is a dictonary of clusters, such that each contains a list of row indices.

#### Plot clusters

We also need a method for evaluating the choice of clusters (k), and visualise the resulting clusters with heatmaps.

https://medium.com/analytics-vidhya/how-to-determine-the-optimal-k-for-k-means-708505d204eb

Retrieving both the medoids and clusters is straightforwards, they can be passed as an array to iloc-based row slicer.

#### Silhouette score

In [15]:
def get_cluster_dfs(clusters):
    cl_dfs = {}
    for i in range(len(clusters)):
        cl_dfs[i] = factors.iloc[clusters[i], :]
    return cl_dfs


def get_all_cluster_distances(clusters, matrix_df):
    """
    Split the all x all distance matrix into separate matrices
    rows = one cluster, first index
    columns = other clurster, second index
    """
    dist_dfs = {}
    for i in range(len(clusters)):
        dist_dfs[i] = {}
        for j in range(len(clusters)):
            dist_dfs[i][j] = matrix_df.iloc[clusters[i], clusters[j]]
    return dist_dfs
    
def get_point_to_cluster_distance(clusters, dist_dfs):

    # get the distances between all points in clusters L and all points in cluster M
    dist_a = {}
    for l in range(len(clusters)):
        dist_a[l] = {}
        k_cl_len = len(clusters[l])
        for m in range(len(clusters)): 
            dist_a[l][m] = []
            # if l == m, we are looking at self distances
            if m == l:
                self_clust_len = len(clusters[l])
                for v in clusters[l]:
                    # going over an array now
                    # the v indexing will be the same within the cluster distance matrix
                    dist_a[l][m].append(dist_dfs[l][m].loc[v].sum() / (self_clust_len-1))
            else:
                # cross-cluster distances now, between point in cluster L towards those in M
                far_clust_len = len(clusters[m])
                for v in clusters[l]:
                    # going over an array now
                    # the v indexing will be the same within the cluster distance matrix
                    dist_a[l][m].append(dist_dfs[l][m].loc[v].sum() / (far_clust_len - 1))
    
    return dist_a

In [16]:
def get_silhouette_score(clusters, d_self):
    sil_scores = {}
    all_clusters = list(clusters.keys())
    
    for i in all_clusters:
        if len(clusters[i]) == 1:
            sil_scores[i] = [0]
        else:
            sil_scores[i] = []
            for v in range(len(clusters[i])):
                a_v = d_self[i][i][v]
                b_v = min([d_self[i][j][v] for j in all_clusters if j != i])
                sil_scores[i].append((b_v - a_v )/ max(a_v, b_v))
                
    avg_sil_scores = {i : sum(sil_scores[i]) / len(sil_scores[i]) for i in all_clusters}
    total_sil = sum(sum(sil_scores[i]) for i in all_clusters)
    total_avg_sil = total_sil / sum(len(sil_scores[i]) for i in all_clusters)
    
    return sil_scores, avg_sil_scores, total_avg_sil

In [17]:
def optimise_number_of_clusters(k_range, matrix_df, matrix, tmax):
    clusters = {}
    medoids = {}
    silhouettes = {}
    
    for k in k_range:
        medoids[k], clusters[k] = kMedoids(matrix, k, tmax)
        dist_dfs = get_all_cluster_distances(clusters[k], matrix_df)
        dist_a = get_point_to_cluster_distance(clusters[k], dist_dfs)
        silhouettes[k] = get_silhouette_score(clusters[k], dist_a)
    
    return clusters, medoids, silhouettes

In [ ]:
k_range = [2,3,4,5,8,10,13,16,18,20]
c1, m1, s1 = optimise_number_of_clusters(k_range, gower_df, gower_matrix, 25)

2 2
3 2
4 1
5 2
8 1
10 2
13 2


https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3848038/

In [ ]:
avg_s = [s1[i][2] for i in k_range]

plt.plot(initial_ks, avg_s)
plt.show()

It appears that a smaller number of clusters is better here. Therefore, let's run multiple repeats for 2-5 clusters.

In [ ]:
k_range = [2,3,4,5]
c1, m1, s1 = optimise_number_of_clusters(k_range, gower_df, gower_matrix, 100)
c2, m2, s2 = optimise_number_of_clusters(k_range, gower_df, gower_matrix, 100)
c3, m3, s3 = optimise_number_of_clusters(k_range, gower_df, gower_matrix, 100)
c4, m4, s4 = optimise_number_of_clusters(k_range, gower_df, gower_matrix, 100)
c5, m5, s5 = optimise_number_of_clusters(k_range, gower_df, gower_matrix, 100)